In [1]:
import os 
import pandas as pd
import numpy as np

from sklearn import preprocessing
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
import sklearn.linear_model

import scipy
from scipy import stats
from scipy.optimize import curve_fit

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [2]:
root_dir = "/Volumes/TOB_WD2/Image_Analysis/CCB02/Dataframes" + "/"

# load data

df = pd.read_csv(root_dir + "MasterDataFrame_CCB02_gTub_for_stats.csv")

In [3]:
df.shape

(157, 32)

In [4]:
df.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Cell_ID', 'Average_Pole_gTub',
       'IntegratedDensity_Pole_gTub', 'Experiment', 'Average_Total_gTub',
       'Minimum_Total_gTub', 'IntegratedDensity_Total_gTub', 'Cell_Volume_um3',
       'Cell_Surface_Area_um2', 'Centrosome_Volume_um3', 'Target',
       'Spindle_Angle_Degrees', 'Spindle_Length_um', 'Spindle_Volume_um3',
       'Spindle_Width_Avg_um', 'Tubulin_Spindle_Average_Intensity',
       'Chromatin_Volume_um3', 'MetaphasePlate_Length_um',
       'MetaphasePlate_Width_um', 'Version', 'Cell_line', 'Condition',
       'Average_Pole_gTub_BGsub', 'Average_Total_gTub_BGsub',
       'Average_gTub_Poles_norm', 'gTub_mass_poles_norm',
       'gTub_mass_at_poles_percent', 'Centrosome_Occupancy',
       'Centrosome_to_spindle_ratio', 'Spindle_Occupancy'],
      dtype='object')

In [5]:
# ANOVA Testing
# ANOVA as generalized linear model (GLM):
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols
import pingouin as pg
import scipy.stats as stats
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.multicomp import MultiComparison

measurement = 'gTub_mass_at_poles_percent'
group_variable = 'Condition'
nan_elements = df[measurement].isnull()
data = df[~nan_elements]

print(data[group_variable].value_counts())

# Bartlett's test for equal variances (One-way ANOVA requires equal variances!)

BartlettResult = stats.bartlett(data[data.Condition == '1_DMSO'][measurement], data[data.Condition == '4_5µM'][measurement], data[data.Condition == '5_10µM'][measurement], data[data.Condition == '6_N2B27+RA'][measurement])

print("The Bartlett test for equal variances of {}: ".format(measurement) + str(BartlettResult))


#results = ols('Aspect-Ratio~ C('+group_variable+')', data=data).fit()
results = ols(measurement + '~ C('+group_variable+')', data = data).fit()
print(results.summary())

aov_table = sm.stats.anova_lm(results, typ = 2)

def anova_table(aov):
    aov['mean_sq'] = aov[:]['sum_sq'] / aov[:]['df']
    
    aov['eta_sq'] = aov[:-1]['sum_sq'] / sum(aov['sum_sq'])
    
    aov['omega_sq'] = (aov[:-1]['sum_sq']-(aov[:-1]['df'] * aov['mean_sq'][-1])) / (sum(aov['sum_sq']) + aov['mean_sq'][-1])
    
    cols = ['sum_sq', 'df', 'mean_sq', 'F', 'PR(>F)', 'eta_sq', 'omega_sq']
    aov = aov[cols]
    return aov

aov_table = anova_table(aov_table)
print("\n ANOVA TABLE: ")
print(aov_table)

# Post-hoc testing
mc = MultiComparison(data[measurement], data[group_variable])
mc_results = mc.tukeyhsd()
print("\n\n POST-HOC testing for {}: \n".format(measurement))
print(mc_results)
print("If \"reject\" = True, then H0 should be rejected")

# Welch's ANOVA when variances are unequal
aov_table_WELCH = pg.welch_anova(dv = measurement, between = group_variable, data = data)
print("\n Welch's ANOVA table") 
print(aov_table_WELCH)

# Post-hoc testing using Games-Howell post-hoc test
mc_results_GamesHowell = pg.pairwise_gameshowell(dv = measurement, between = group_variable, data = data)
print("\n Games-Howell post-hoc test table") 
print(mc_results_GamesHowell)

5_10µM        49
4_5µM         45
1_DMSO        39
6_N2B27+RA    24
Name: Condition, dtype: int64
The Bartlett test for equal variances of gTub_mass_at_poles_percent: BartlettResult(statistic=9.625555355311999, pvalue=0.022032500955905015)
                                OLS Regression Results                                
Dep. Variable:     gTub_mass_at_poles_percent   R-squared:                       0.153
Model:                                    OLS   Adj. R-squared:                  0.136
Method:                         Least Squares   F-statistic:                     9.202
Date:                        Mon, 10 Jul 2023   Prob (F-statistic):           1.24e-05
Time:                                17:29:39   Log-Likelihood:                -241.40
No. Observations:                         157   AIC:                             490.8
Df Residuals:                             153   BIC:                             503.0
Df Model:                                   3                   

/anaconda3/envs/statistics/lib/python3.8/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.4.0, the latest is 0.5.3.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(
/anaconda3/envs/statistics/lib/python3.8/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package outdated is out of date. Your version is 0.2.1, the latest is 0.2.2.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(
